In [1]:
from bs4 import BeautifulSoup
import re
import json
from datasets import Dataset
import torch
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import pandas as pd 
import yfinance as yf 
from yahoofinancials import YahooFinancials
from datetime import datetime

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
urbn_soup = BeautifulSoup(open(r"\Users\User\Desktop\new_model\stock_price_and_news\stock_news\dataset\urbn_final\urbn_2003.html", encoding="utf8"),"html.parser")


In [3]:
urbn_timestamp = []
for e in urbn_soup.find_all("span", {"class": "article__timestamp"}):
    urbn_timestamp.append(e.get_text())


In [4]:
urbn_file = []
for urbn_text in urbn_soup.find_all("h3"):
    urbn_file.append(urbn_text.get_text())
urbn_clean1 = []
for urbn_text1 in urbn_file:
    urbn_clean1.append(urbn_text1.replace("\n", ""))
urbn_remove = []
for e in urbn_clean1:
    if e not in (
 'No Recent Tickers',
 'Overview'
 ):
        urbn_remove.append(e)
urbn_space = []
for e in urbn_remove:
    urbn_space.append(e.strip())
urbn_fullstop = []
for e in urbn_space:
    urbn_fullstop.append(e.replace(".",""))
urbn_q = []
for e in urbn_fullstop:
    urbn_q.append(e.replace("?",""))
urbn_colon = []
for e in urbn_q:
    urbn_colon.append(e.replace(":",""))
urbn_comma = []
for e in urbn_colon:
    urbn_comma.append(e.replace(",",""))
urbn_percent = []
for e in urbn_comma:
    urbn_percent.append(e.replace("%","percent"))
urbn_lower = []
for i in range(len(urbn_percent)):
    urbn_lower.append(urbn_percent[i].lower())
urbn_barr = []
for e in urbn_lower:
    urbn_barr.append(e.replace("barron's",""))
urbn_pain = []
for e in urbn_barr:
    if e not in ( 'marketwatch',
 'company',
 'dow jones network'):
        urbn_pain.append(e)
        

In [5]:
urbn_pain[-10:]

['urban outfitters (urbn) q4 earnings miss sales improve y/y',
 'analysts offer insights on consumer goods companies ross stores (rost) and urban outfitters (urbn)',
 'urban outfitters (urbn) q4 earnings miss estimates',
 '5 key earnings charts this week',
 'urban outfitters (urbn) to post q4 earnings things to note',
 '7 luxury apparel stocks that will strut their way through 2022',
 'hyperion capital advisors lp buys sotera health co american eagle outfitters inc urban ',
 'turtle creek asset management inc buys euronet worldwide inc alliance data systems corp ',
 'analysts offer insights on consumer goods companies cvs health (cvs) mohawk (mhk) and urban outfitters (urbn)',
 "urban outfitters' (urbn) q4 sales increase on brand strength"]

In [6]:
device = torch.device("cpu")

In [7]:
from transformers import BertModel, BertTokenizer,AutoModelForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1)[:, None])
    return e_x / np.sum(e_x, axis=1)[:, None]

In [9]:
label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}

urbn_out = []
urbn_score = []
for i in range(0, len(urbn_pain), 10):
    tokenized_urbn = tokenizer(urbn_pain[i:i+10], truncation = True, padding = True, return_tensors = "pt")
    urbn_output = model(tokenized_urbn['input_ids'])
    soft_logits_urbn = softmax(np.array(urbn_output.logits.detach().cpu()))
    sentiment_score_urbn = soft_logits_urbn[:,0]-soft_logits_urbn[:,1]
    urbn_pred = np.squeeze(np.argmax(soft_logits_urbn, axis = 1))
    urbn_out.extend(urbn_pred)
    urbn_score.extend(sentiment_score_urbn)
#print(gps_out) 
print(sentiment_score_urbn)

[-0.00750959 -0.00627327]


In [10]:
print(len(urbn_out))
urbn_class = [label_dict[pred] for pred in urbn_out]
print(len(urbn_class))
#print(gps_class)

1312
1312


In [11]:
urbn_stock_price  = yf.download('urbn', start='2003-09-23', end='2022-04-03', time_interval = 'daily')


[*********************100%***********************]  1 of 1 completed


In [12]:
urbn_stock_price = urbn_stock_price.drop(['Open', 'High','Low','Volume', 'Adj Close'], axis = 1)


In [13]:
urbn_et = []
for e in urbn_timestamp:
    urbn_et.append(e.replace("ET",""))
urbn_at1 = []
for e in urbn_et:
    urbn_at1.append(e.replace("at",""))
urbn_pm1 = []
for e in urbn_at1:
    urbn_pm1.append(e.replace("p.m.",""))
urbn_am1 = []
for e in urbn_pm1:
    urbn_am1.append(e.replace("a.m.",""))
urbn_commas = []
for e in urbn_am1:
    urbn_commas.append(e.replace(",",""))
urbn_fullstops = []
for e in urbn_commas:
    urbn_fullstops.append(e.replace(".",""))
urbn_timestamp_final = urbn_fullstops

In [14]:
urbn_timestamp_final[-5:]


['Feb 22 2022  1:46  ',
 'Feb 18 2022  7:38  ',
 'Feb 15 2022  10:38  ',
 'Feb 13 2022  9:55  ',
 'Feb 11 2022  3:38  ']

In [15]:
urbn_date = [datetime.strptime(x,'%b %d %Y  %I:%M ') for x in urbn_timestamp_final]


In [16]:
urbn_date[-5:]

[datetime.datetime(2022, 2, 22, 1, 46),
 datetime.datetime(2022, 2, 18, 7, 38),
 datetime.datetime(2022, 2, 15, 10, 38),
 datetime.datetime(2022, 2, 13, 9, 55),
 datetime.datetime(2022, 2, 11, 3, 38)]

In [17]:
urbn_score = pd.DataFrame (urbn_score, columns = ['sentiment_score'])
scaler = MinMaxScaler(feature_range=(0, 1))
urbn_senti_scaler = scaler.fit_transform(urbn_score.values.reshape(-1,1))
urbn_date = pd.DataFrame (urbn_date, columns = ['Date'])
urbn_senti_scaler = pd.DataFrame (urbn_senti_scaler, columns = ['sentiment_score'])

urbn_news_table = pd.concat([urbn_date, urbn_senti_scaler], axis=1)

In [18]:
urbn_news_table.isnull().values.any()


False

In [19]:
urbn_news_table

,Date,sentiment_score
0,2022-04-01 02:29:00,0.536092
1,2022-02-15 00:53:00,0.178522
2,2022-01-11 08:28:00,0.677281
3,2022-01-11 08:10:00,0.504616
4,2022-01-11 08:10:00,0.553535
...,...,...
1307,2022-02-22 01:46:00,0.594336
1308,2022-02-18 07:38:00,0.645936
1309,2022-02-15 10:38:00,0.621654
1310,2022-02-13 09:55:00,0.685893


In [20]:
urbn_news = urbn_news_table.sort_values(by ="Date")

In [21]:
scaler = MinMaxScaler(feature_range=(0, 1))
urbn_scaler = scaler.fit_transform(urbn_stock_price.values.reshape(-1,1))

In [22]:
urbn_pure_price = pd.DataFrame(urbn_scaler, columns = ['Closing_price'])
urbn_pure_price = urbn_pure_price.reset_index()
#urbn_pure_price = urbn_pure_price.drop(columns= ['Date'])

urbn_stock_price1 = urbn_stock_price.reset_index()
urbn_stockprice_date = urbn_stock_price1["Date"]
urbn_stockprice_date = pd.DataFrame({ 'Stock_price_date':urbn_stockprice_date.values})


In [23]:
urbn_stockprice_date

,Stock_price_date
0,2003-09-22
1,2003-09-23
2,2003-09-24
3,2003-09-25
4,2003-09-26
...,...
4661,2022-03-28
4662,2022-03-29
4663,2022-03-30
4664,2022-03-31


In [24]:
urbn_stock_table =pd.concat([urbn_stockprice_date,urbn_pure_price], axis=1)
print(urbn_stock_table.isnull().values.any())
urbn_stock_table1 = urbn_stock_table.set_index('Stock_price_date')
urbn_stock_table1 = urbn_stock_table1.drop(['index'],axis= 1)

False


In [25]:
urbn_news_table2 = urbn_news_table.sort_values("Date")
urbn_news_table2.head(10)

,Date,sentiment_score
1291,2003-09-26 10:15:00,0.085076
1290,2003-10-02 10:37:00,0.282076
1289,2003-10-21 10:47:00,0.074614
1288,2004-04-06 09:43:00,0.361402
1287,2004-05-28 09:49:00,0.491611
1286,2004-10-28 09:27:00,0.398118
1285,2004-11-05 09:19:00,0.269345
1284,2004-11-09 09:26:00,0.358851
1283,2005-03-02 09:16:00,0.260440
1282,2005-03-11 10:33:00,0.187051


In [26]:
urbn_news_table3 = urbn_news_table2.set_index('Date')

In [27]:
urbn_news_table3

,sentiment_score
Date,
2003-09-26 10:15:00,0.085076
2003-10-02 10:37:00,0.282076
2003-10-21 10:47:00,0.074614
2004-04-06 09:43:00,0.361402
2004-05-28 09:49:00,0.491611
...,...
2022-03-31 11:30:00,0.560864
2022-04-01 02:29:00,0.536092
2022-04-12 10:43:00,0.419552


In [28]:
urbn_final_table = urbn_news_table3.assign(key=urbn_news_table3.index.normalize())\
   .merge(urbn_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)

C:\Users\User\AppData\Local\Temp\ipykernel_25728\4185542038.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .merge(urbn_stock_table1, left_on='key', right_index=True, how='left').drop('key', 1)


In [29]:
urbn_final_table

,sentiment_score,Closing_price
Date,,
2003-09-26 10:15:00,0.085076,0.005820
2003-10-02 10:37:00,0.282076,0.014638
2003-10-21 10:47:00,0.074614,0.046678
2004-04-06 09:43:00,0.361402,0.145209
2004-05-28 09:49:00,0.491611,0.176661
...,...,...
2022-03-31 11:30:00,0.560864,0.444327
2022-04-01 02:29:00,0.536092,0.434921
2022-04-12 10:43:00,0.419552,NaN


In [30]:
urbn_for_lstm = urbn_final_table.dropna()

In [31]:
urbn_for_lstm.isnull().any()

sentiment_score    False
Closing_price      False
dtype: bool

In [32]:
urbn_for_lstm_chakky = urbn_final_table.fillna(method = "ffill")

In [34]:
urbn_for_lstm_chakky = urbn_for_lstm_chakky.fillna(method = "bfill")

In [47]:
urbn_for_lstm_chakky.to_csv('urbn_ffill_bert_new.csv', index = True)
urbn_for_lstm.to_csv('urbn_lstm_bert_new.csv', index = True)